# Using TensorFlow directly

So far we've looked at Keras, a library designed to make experimentation easy by expressing network architectures at a fairly high level. Here, we'll take a brief look at what using TensorFlow is like. We will re-make our [multi-layer, fully-connected example](02-notmnist-mlp-keras.ipynb).

We will see that TensorFlow gives more direct access to the individual variables and computations inside the network. This can be handy for a few reasons:

* Less "magic" -- you see more of what's happening, which may be useful pedagogically when learning, and also when debugging.
* More low-level flexibility -- you can arrange the computation however you want.

That said, I recommend using Keras or another higher level library when you don't need the full flexibility of TF. 

(This code is based on [a tensorflow tutorial](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb))

We will again be using the pickle file of the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html), same as in our Keras example.

In [ ]:
# for DSX, need to switch to the right directory. Detect using path name.
s = %pwd
if s.startswith('/gpfs'):
    %cd ~/deep-learning-workshop/

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# Loading the data

This doesn't use the framework, so is the same as our Keras notebook.

In [2]:
import notmnist
notmnist_path = "~/data/notmnist/notMNIST.pickle"

In [3]:
# the data, shuffled and split between train, validation, and test sets
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = notmnist.load_data(notmnist_path)

In [4]:
x_train = x_train.reshape(-1, 784)
x_valid = x_valid.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 valid samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices

def to_categorical(labels, num_classes):
    """
    Reimplementing the function from Keras np_utils.
    """
    # (labels[:,None] adds an extra dimension, 
    # then numpy projects num_classes to 2D too)
    labels = (np.arange(num_classes) == labels[:,None]).astype(np.float32)
    return labels

nb_classes = 10
image_size = 28
y_train = to_categorical(y_train, nb_classes)
y_valid = to_categorical(y_valid, nb_classes)
y_test = to_categorical(y_test, nb_classes)

In [6]:
# make sure shapes are right
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_test.shape)
print(x_test.shape, y_test.shape)

(50000, 784) (50000, 10)
(10000, 784) (10000, 10)
(10000, 784) (10000, 10)


# Multinomial logistic regression

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [24]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data. 
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(x_train[:train_subset, :])
    tf_train_labels = tf.constant(y_train[:train_subset])
    tf_valid_dataset = tf.constant(x_valid)
    tf_test_dataset = tf.constant(x_test)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, nb_classes]))
    biases = tf.Variable(tf.zeros([nb_classes]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
         tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [25]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [27]:
num_steps = 801


with tf.Session(graph=graph) as session:
      # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, y_train[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), y_valid))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
Loss at step 0: 16.130016
Training accuracy: 9.9%
Validation accuracy: 13.8%
Loss at step 100: 2.303081
Training accuracy: 72.5%
Validation accuracy: 70.9%
Loss at step 200: 1.848797
Training accuracy: 75.6%
Validation accuracy: 73.7%
Loss at step 300: 1.609403
Training accuracy: 77.0%
Validation accuracy: 74.9%
Loss at step 400: 1.448529
Training accuracy: 77.8%
Validation accuracy: 75.2%
Loss at step 500: 1.329500
Training accuracy: 78.4%
Validation accuracy: 75.2%
Loss at step 600: 1.236034
Training accuracy: 78.9%
Validation accuracy: 75.3%
Loss at step 700: 1.159743
Training accuracy: 79.5%
Validation accuracy: 75.5%
Loss at step 800: 1.095791
Training accuracy: 80.0%
Validation accuracy: 75.5%
Test accuracy: 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [17]:
batch_size = 128

In [30]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, nb_classes))
    tf_valid_dataset = tf.constant(x_valid)
    tf_test_dataset = tf.constant(x_test)
    
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, nb_classes]))
    biases = tf.Variable(tf.zeros([nb_classes]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)
      

Let's run it:

In [33]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = x_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), y_valid))
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(), y_test))

Initialized
Minibatch loss at step 0: 17.644871
Minibatch accuracy: 6.2%
Validation accuracy: 13.8%
Minibatch loss at step 500: 1.320502
Minibatch accuracy: 75.8%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.818259
Minibatch accuracy: 81.2%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 1.194087
Minibatch accuracy: 77.3%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 1.157571
Minibatch accuracy: 76.6%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 1.027862
Minibatch accuracy: 74.2%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 0.824554
Minibatch accuracy: 84.4%
Validation accuracy: 78.5%
Test accuracy: 86.1%


# Multi-layer example

Now we'll make the same model as model3 from [our Keras example](02-notmnist-mlp-keras.ipynb) -- two dense layers, with dropout. We'll reorganize our code to be a bit neater while we're at it.

(Code structure roughly based on tensorflow mnist, cifar10 examples. Thanks!)

In [7]:
# Constants
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
NUM_LABELS = 10

_DEBUG = False # if true, print more

def dprint(*args):
    """
    print, but only if _DEBUG.
    """
    if _DEBUG:
        print(*args)


In [8]:
def placeholder_inputs():
    """
    Returns:
        tuple (images_placeholder, labels_placeholder, keep_prob)
    """
    with tf.name_scope('input'):
        images_placeholder = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS), name="images")
        labels_placeholder = tf.placeholder(tf.float32, shape=(None, NUM_LABELS), name="labels")

        # use a placeholder so we can use 1.0 (no dropout) for test/validation, and a lower value
        # for training
        keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        tf.summary.scalar('dropout_keep_probability', keep_prob)

    return (images_placeholder, labels_placeholder, keep_prob)


In [9]:
def add_variable_summaries(var, name):
    """
    Attach summaries to a Tensor, for use in Tensorboard: mean, stddev, max, min, histogram.

    Args:
        var: a tensor
        name: name to use for the summary
    """
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('sttdev/' + name, stddev)
        tf.summary.scalar('max/' + name, tf.reduce_max(var))
        tf.summary.scalar('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

In [10]:
def inference(images, keep_prob, **kwargs):
    """
    Inference for a deeper network with two layers.

    Args:
        images: 2D tensor [batch_size, IMAGE_SIZE*IMAGE_SIZE]
            (grayscale images, so no depth dimension). May be a placeholder or constant.
        keep_prob: for dropout, probability of keeping an activation -- for validation and
            test, set to 1.0 for no dropout.

    Returns:
        logits: tf object that corresponds to the logits for the images: [batch_size, NUM_LABELS]
    """
    beta = 0.001  # how much regularization to do
    keep_prob = 0.5  # if using dropout
    hidden_units = [256, 128, NUM_LABELS]

    def make_hidden_layer(name, input_tensor, output_size, last_layer=False):
        """
        If last_layer=True, make logits, without a relu
        """
        if isinstance(input_tensor, np.ndarray):
            input_size = input_tensor.shape[1]
        else:
            # otherwise, should be a TF Tensor.
            input_size = int(input_tensor.get_shape()[1])
        dprint("creating hidden layer with input_size {}, output_size {}".format(input_size, output_size))
        with tf.variable_scope(layer_name):
            weights = tf.get_variable("weights",
                                  initializer=tf.truncated_normal([input_size, output_size]),
                                  regularizer=tf.contrib.layers.l2_regularizer(beta))
            # no regularizer for biases
            biases = tf.get_variable("biases", initializer=tf.zeros([output_size]))
            if not last_layer:
                hidden = tf.nn.relu(tf.matmul(input_tensor, weights) + biases, name="hidden")
                dropped = tf.nn.dropout(hidden, keep_prob)
                return dropped
            else:
                dprint("Last layer. Creating logits instead of a relu.")
                logits = tf.matmul(layer_input, weights) + biases
                return logits

    layer_input = images
    for i, layer_size in enumerate(hidden_units):
        layer_name = "hidden_"+str(i)
        is_last_layer = (i == len(hidden_units)-1)
        layer_input = make_hidden_layer(layer_name, layer_input, layer_size, is_last_layer)

    return layer_input  # logits

In [11]:
def loss(logits, labels_placeholder):
    """
    Compute the loss for the given logits, labels.
    """
    with tf.name_scope("loss"):
        total_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_placeholder))

        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        if len(reg_losses):
            reg_loss = tf.add_n(reg_losses)
            total_loss += reg_loss
            tf.summary.scalar("regularization loss", reg_loss)

        tf.summary.scalar("total loss", total_loss)
    return total_loss


In [19]:
def train(total_loss, step_counter, initial_learning_rate=0.5):
    """
    Train the network using SGD.

    Args:
        total_loss: tensor from loss()
        global_step: Integer Variable counting steps -- used to reduce learning rate.

    Returns:
        train_op: op for training
    """
    # initial * decay_rate^(global_step / decay_steps)
    decay_steps = 10000   # smaller means faster
    decay_rate = 0.96     # smaller means faster
    with tf.name_scope("train"):
        learning_rate = tf.train.exponential_decay(initial_learning_rate, step_counter, 10000, 0.96)
        tf.summary.scalar("learning rate", learning_rate)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
            total_loss, global_step=step_counter)

    return optimizer


In [12]:
def predict(logits):
    """
    Return the prediction given logits.
    """
    with tf.name_scope("output"):
        return tf.nn.softmax(logits)


In [13]:
def batch_data_and_labels(step, batch_size, train_dataset, train_labels):
    """
    Get data for training a batch. Assumes the dataset has been randomized.

    Returns:
        tuple (batch_data, batch_labels)
    """
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    return batch_data, batch_labels



In [22]:
def run_experiment(data, batch_size, num_steps,
                  inference_fn,
                  initial_learning_rate=0.5,
                  dropout=False):
    """
    Helper function to train and test a network. Makes a new graph, session.

    Args:
        data: {'train': (data, labels), 'valid': ..., 'test': ...}
        batch_size, num_steps: parameters
        inference_fn: function: images->logits. Defines network structure.
        initial_learning_rate: pass to the optimizer
        dropout: if True, use dropout. inference_fn has to take a
             dropout param.
    """

    # Create the graph
    graph = tf.Graph()
    with graph.as_default():
        step_counter = tf.Variable(0, name="step_counter")  # used to decay the learning rate
        images_placeholder, labels_placeholder, keep_prob = placeholder_inputs()
        # for training
        logits = inference_fn(images_placeholder, keep_prob=keep_prob)
        total_loss = loss(logits, labels_placeholder)
        train_op = train(total_loss, step_counter,
                           initial_learning_rate=initial_learning_rate)
        prediction = predict(logits)

        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(labels_placeholder, 1), tf.argmax(prediction, 1))
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)


    def feed_dict(which_data, step=None):
        """
        Args:
            which_data: str -- either 'train', 'valid', or 'test'. If train, use the real
                dropout probability and the next batch of training data. Otherwise, no
                dropout, and appropriate data.
            step: for 'train' mode: which step is this, for getting the next batch of data.
        Returns:
            dict mapping data into placeholders.
        """
        if which_data == 'train':
            batch_data, batch_labels = batch_data_and_labels(
                step, batch_size, *data['train'])

            # What values to feed to the placeholders?
            feed_dict = {images_placeholder : batch_data,
                         labels_placeholder : batch_labels,
                         keep_prob: 0.5}
        else:
            x, y = data[which_data]
            feed_dict = {images_placeholder : x,
                         labels_placeholder : y,
                         keep_prob: 1.0}
        return feed_dict


    with tf.Session(graph=graph) as session:
        merged = tf.summary.merge_all()
        # run 'tensorboard --logdir=./logs/tensorflow-demo'
        writer = tf.summary.FileWriter("./logs/tensorflow-demo", graph)
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):

            # run the network to train
            summary, _, batch_loss, batch_accuracy = session.run(
              [merged, train_op, total_loss, accuracy], feed_dict=feed_dict('train', step))
            writer.add_summary(summary, step)

            # blow up if loss is nan
            if np.isnan(batch_loss):
                raise ValueError("Batch loss is NaN! (Try lowering learning rate)")

            if (step) % 500 == 0 or (step+1)==num_steps:
                # Compute validation and test accuracy, stats
                summary, valid_accuracy = session.run([merged, accuracy], feed_dict('valid'))
                writer.add_summary(summary, step)
                print()
                print("Minibatch loss at step %d: %f" % (step, batch_loss))
                print("Minibatch accuracy: %.3f" % batch_accuracy)
                print("Validation accuracy: %.3f" % valid_accuracy)

        summary, test_accuracy = session.run([merged, accuracy], feed_dict('test'))
        writer.add_summary(summary, step)
        print("\nTest accuracy: %.3f" % test_accuracy)


In [15]:
data = {"train": (x_train, y_train),
        "valid": (x_valid, y_valid),
        "test": (x_test, y_test)}

In [23]:
run_experiment(data, batch_size=batch_size, num_steps=30000,
              inference_fn=inference, dropout=True,
              initial_learning_rate=0.1)

INFO:tensorflow:Summary name regularization loss is illegal; using regularization_loss instead.
INFO:tensorflow:Summary name total loss is illegal; using total_loss instead.
INFO:tensorflow:Summary name learning rate is illegal; using learning_rate instead.
Initialized

Minibatch loss at step 0: 2441.237549
Minibatch accuracy: 0.094
Validation accuracy: 0.174

Minibatch loss at step 500: 85.890900
Minibatch accuracy: 0.102
Validation accuracy: 0.111

Minibatch loss at step 1000: 77.978088
Minibatch accuracy: 0.086
Validation accuracy: 0.111

Minibatch loss at step 1500: 70.709419
Minibatch accuracy: 0.141
Validation accuracy: 0.108

Minibatch loss at step 2000: 64.264053
Minibatch accuracy: 0.117
Validation accuracy: 0.108

Minibatch loss at step 2500: 58.552341
Minibatch accuracy: 0.086
Validation accuracy: 0.107

Minibatch loss at step 3000: 53.128201
Minibatch accuracy: 0.102
Validation accuracy: 0.110

Minibatch loss at step 3500: 48.335041
Minibatch accuracy: 0.078
Validation accu

One difference between this and our Keras code is that here we use regular SGD (stochastic gradient descent), rather than the RMSprop optimizer we used with Keras. That makes training a bit slower, so we get lower accuracy, even after training a bit longer. 

Let's look at our tensorboard output:

## The compute graph
Looks neater than with Keras -- we can control the grouping ourselves.
<img src="img/tf-demo-tb-graph.png" width="400px">

## Accuracy:
<img src="img/tf-demo-accuracy.png" width="400px">

## Losses
Our model uses regularization to encourage weights to be smaller, and our code tracks that part of the loss separately.
<img src="img/tf-demo-losses.png" width="400px">
